# Qualicum Beach Orthomosaic Processing with GCPs

This notebook processes drone imagery to create orthomosaics using Agisoft Metashape, comparing results with and without ground control points (GCPs).

## Workflow:
1. Load GCPs from KMZ file
2. Download drone imagery from S3 (all 12 cells)
3. Process orthomosaic WITHOUT GCPs
4. Process orthomosaic WITH GCPs
5. Compare both orthomosaics against ESRI and OpenStreetMap basemaps
6. Generate comprehensive quality report


## Setup and Imports


In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Add package to path
package_dir = Path.cwd()
sys.path.insert(0, str(package_dir))

from qualicum_beach_gcp_analysis import (
    load_gcps_from_kmz,
    download_basemap,
    visualize_gcps_on_basemap,
    calculate_gcp_bbox,
    download_all_images_from_input_dir,
    export_to_metashape_csv,
    export_to_metashape_xml,
    process_orthomosaic,
    PhotoMatchQuality,
    DepthMapQuality,
    compare_orthomosaic_to_basemap,
    generate_comparison_report,
    generate_markdown_report,
)

print("✓ Imports successful!")


✓ Imports successful!


## Step 1: Load Ground Control Points


In [2]:
# Path to the KMZ file
kmz_path = "/Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz"

# Load GCPs
gcps = load_gcps_from_kmz(kmz_path)

print(f"\n✓ Loaded {len(gcps)} ground control points")

# Display all GCPs
if gcps:
    print("\nGCPs:")
    for i, gcp in enumerate(gcps):
        print(f"  {i+1:2d}. {gcp.get('id', 'Unknown'):20s}: ({gcp['lat']:.6f}, {gcp['lon']:.6f}, z={gcp.get('z', 0):.2f})")
else:
    print("\n⚠️  No GCPs found!")


Loading GCPs from: /Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz
Found 1 KML file(s) in KMZ
Attempting to fix namespace issues...
✓ Fixed namespace issues in KML file
Found 12 placemarks in KMZ file (namespace: http://www.opengis.net/kml/2.2)
Successfully parsed 12 GCPs from KMZ file

✓ Loaded 12 ground control points

GCPs:
   1. 8928d89ac03ffff     : (49.352544, -124.407904, z=0.00)
   2. 8928d89ac0bffff     : (49.354342, -124.404136, z=0.00)
   3. 8928d89ac1bffff     : (49.351585, -124.403857, z=0.00)
   4. 8928d89ac43ffff     : (49.355182, -124.396319, z=0.00)
   5. 8928d89ac47ffff     : (49.356141, -124.400367, z=0.00)
   6. 8928d89ac53ffff     : (49.352425, -124.396040, z=0.00)
   7. 8928d89ac57ffff     : (49.353384, -124.400088, z=0.00)
   8. 8928d89ac5bffff     : (49.354224, -124.392271, z=0.00)
   9. 8928d89ac73ffff     : (49.357100, -124.404415, z=0.00)
  10. 8928d89acc7ffff     : (49.348828, -12

## Step 2: Calculate Bounding Box and Download Reference Basemaps


In [3]:
# Create output directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Calculate bounding box
bbox = calculate_gcp_bbox(gcps, padding=0.01)
min_lat, min_lon, max_lat, max_lon = bbox

print(f"Bounding box:")
print(f"  Latitude: {min_lat:.6f} to {max_lat:.6f}")
print(f"  Longitude: {min_lon:.6f} to {max_lon:.6f}")

# Calculate center latitude for resolution calculation
center_lat = (min_lat + max_lat) / 2.0

# Import math and rasterio for resolution calculations
import math
import rasterio

# Function to calculate resolution in meters per pixel for a given zoom level
def calculate_resolution_meters_per_pixel(zoom: int, latitude: float) -> float:
    """Calculate resolution in meters per pixel for a given zoom level and latitude."""
    # Earth's circumference at equator in meters
    earth_circumference = 40075017.0  # meters
    # Resolution at equator for given zoom level
    resolution_equator = earth_circumference / (256 * (2 ** zoom))
    # Adjust for latitude (pixels get smaller as you move away from equator)
    resolution = resolution_equator * math.cos(math.radians(latitude))
    return resolution

# Download current basemaps at default zoom to check resolution
print("\n" + "="*60)
print("Step 2a: Download current basemaps and check resolution")
print("="*60)

basemap_esri_path_low = str(output_dir / "qualicum_beach_basemap_esri_lowres.tif")
if Path(basemap_esri_path_low).exists():
    print(f"\n✓ ESRI basemap (low res) already exists: {basemap_esri_path_low}")
    print("  REUSING existing file (skipping download)")
else:
    print("\nDownloading ESRI World Imagery basemap (low resolution for comparison)...")
    basemap_esri_path_low = download_basemap(
        bbox=bbox,
        output_path=basemap_esri_path_low,
        source="esri_world_imagery",
        zoom=None  # Will use default zoom (typically 13)
    )

# Get actual zoom level used and calculate resolution
with rasterio.open(basemap_esri_path_low) as src:
    esri_width = src.width
    esri_height = src.height
    esri_bounds = src.bounds
    # Calculate resolution from actual bounds and dimensions
    width_meters = (esri_bounds.right - esri_bounds.left) * 111320 * math.cos(math.radians(center_lat))
    height_meters = (esri_bounds.top - esri_bounds.bottom) * 111320
    esri_resolution_x = width_meters / esri_width
    esri_resolution_y = height_meters / esri_height
    esri_resolution = (esri_resolution_x + esri_resolution_y) / 2.0

print(f"  Dimensions: {esri_width} x {esri_height} pixels")
print(f"  Resolution: {esri_resolution:.2f} meters per pixel")

basemap_osm_path_low = str(output_dir / "qualicum_beach_basemap_osm_lowres.tif")
if Path(basemap_osm_path_low).exists():
    print(f"\n✓ OpenStreetMap basemap (low res) already exists: {basemap_osm_path_low}")
    print("  REUSING existing file (skipping download)")
else:
    print("\nDownloading OpenStreetMap basemap (low resolution for comparison)...")
    basemap_osm_path_low = download_basemap(
        bbox=bbox,
        output_path=basemap_osm_path_low,
        source="openstreetmap",
        zoom=None  # Will use default zoom (typically 13)
    )

with rasterio.open(basemap_osm_path_low) as src:
    osm_width = src.width
    osm_height = src.height
    osm_bounds = src.bounds
    width_meters = (osm_bounds.right - osm_bounds.left) * 111320 * math.cos(math.radians(center_lat))
    height_meters = (osm_bounds.top - osm_bounds.bottom) * 111320
    osm_resolution_x = width_meters / osm_width
    osm_resolution_y = height_meters / osm_height
    osm_resolution = (osm_resolution_x + osm_resolution_y) / 2.0

print(f"  Dimensions: {osm_width} x {osm_height} pixels")
print(f"  Resolution: {osm_resolution:.2f} meters per pixel")

print("\n" + "="*60)
print("Current Basemap Resolution Summary:")
print("="*60)
print(f"ESRI World Imagery:     {esri_resolution:.2f} m/pixel")
print(f"OpenStreetMap:          {osm_resolution:.2f} m/pixel")
print(f"\n⚠️  These resolutions are too low for detailed analysis!")

# Calculate target zoom level for higher resolution
# Target: ~0.5-1.0 meters per pixel (suitable for detailed comparison)
target_resolution = 0.5  # meters per pixel
target_zoom = None
for zoom in range(19, 12, -1):
    res = calculate_resolution_meters_per_pixel(zoom, center_lat)
    if res <= target_resolution:
        target_zoom = zoom
        break

if target_zoom is None:
    target_zoom = 18  # Fallback to zoom 18

print(f"\nTarget resolution: {target_resolution} m/pixel")
print(f"Calculated zoom level: {target_zoom}")
print(f"Expected resolution at zoom {target_zoom}: {calculate_resolution_meters_per_pixel(target_zoom, center_lat):.2f} m/pixel")

print("\n" + "="*60)
print("Step 2b: Download HIGH RESOLUTION basemaps")
print("="*60)

# Download HIGH RESOLUTION ESRI World Imagery basemap
basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri.tif")
if Path(basemap_esri_path).exists():
    print(f"\n✓ ESRI basemap (HIGH RES) already exists: {basemap_esri_path}")
    print("  REUSING existing file (skipping download)")
else:
    print(f"\nDownloading HIGH RESOLUTION ESRI World Imagery basemap (zoom {target_zoom})...")
    basemap_esri_path = download_basemap(
        bbox=bbox,
        output_path=basemap_esri_path,
        source="esri_world_imagery",
        zoom=target_zoom  # Use high zoom for better resolution
    )

with rasterio.open(basemap_esri_path) as src:
    esri_hr_width = src.width
    esri_hr_height = src.height
    esri_hr_bounds = src.bounds
    width_meters = (esri_hr_bounds.right - esri_hr_bounds.left) * 111320 * math.cos(math.radians(center_lat))
    height_meters = (esri_hr_bounds.top - esri_hr_bounds.bottom) * 111320
    esri_hr_resolution = ((width_meters / esri_hr_width) + (height_meters / esri_hr_height)) / 2.0

print(f"  Dimensions: {esri_hr_width} x {esri_hr_height} pixels")
print(f"  Resolution: {esri_hr_resolution:.2f} meters per pixel")
if not Path(basemap_esri_path).exists() or Path(basemap_esri_path).stat().st_mtime > Path(basemap_esri_path_low).stat().st_mtime:
    print(f"  Improvement: {esri_resolution/esri_hr_resolution:.1f}x higher resolution")

# Download HIGH RESOLUTION OpenStreetMap basemap
basemap_osm_path = str(output_dir / "qualicum_beach_basemap_osm.tif")
if Path(basemap_osm_path).exists():
    print(f"\n✓ OpenStreetMap basemap (HIGH RES) already exists: {basemap_osm_path}")
    print("  REUSING existing file (skipping download)")
else:
    print(f"\nDownloading HIGH RESOLUTION OpenStreetMap basemap (zoom {target_zoom})...")
    basemap_osm_path = download_basemap(
        bbox=bbox,
        output_path=basemap_osm_path,
        source="openstreetmap",
        zoom=target_zoom  # Use high zoom for better resolution
    )

with rasterio.open(basemap_osm_path) as src:
    osm_hr_width = src.width
    osm_hr_height = src.height
    osm_hr_bounds = src.bounds
    width_meters = (osm_hr_bounds.right - osm_hr_bounds.left) * 111320 * math.cos(math.radians(center_lat))
    height_meters = (osm_hr_bounds.top - osm_hr_bounds.bottom) * 111320
    osm_hr_resolution = ((width_meters / osm_hr_width) + (height_meters / osm_hr_height)) / 2.0

print(f"  Dimensions: {osm_hr_width} x {osm_hr_height} pixels")
print(f"  Resolution: {osm_hr_resolution:.2f} meters per pixel")
if not Path(basemap_osm_path).exists() or Path(basemap_osm_path).stat().st_mtime > Path(basemap_osm_path_low).stat().st_mtime:
    print(f"  Improvement: {osm_resolution/osm_hr_resolution:.1f}x higher resolution")

# Optionally download Google Satellite basemap (high resolution)
# Note: Google tiles may be subject to Terms of Service restrictions
# For production use, consider Google Earth Engine API with authentication
basemap_google_path = str(output_dir / "qualicum_beach_basemap_google.tif")
google_downloaded = False
google_hr_resolution = None

if Path(basemap_google_path).exists():
    print(f"\n✓ Google Satellite basemap (HIGH RES) already exists: {basemap_google_path}")
    print("  REUSING existing file (skipping download)")
    try:
        with rasterio.open(basemap_google_path) as src:
            google_hr_width = src.width
            google_hr_height = src.height
            google_hr_bounds = src.bounds
            width_meters = (google_hr_bounds.right - google_hr_bounds.left) * 111320 * math.cos(math.radians(center_lat))
            height_meters = (google_hr_bounds.top - google_hr_bounds.bottom) * 111320
            google_hr_resolution = ((width_meters / google_hr_width) + (height_meters / google_hr_height)) / 2.0
        print(f"  Dimensions: {google_hr_width} x {google_hr_height} pixels")
        print(f"  Resolution: {google_hr_resolution:.2f} meters per pixel")
        google_downloaded = True
    except Exception as e:
        print(f"  ⚠️  Error reading existing Google basemap: {e}")
        google_downloaded = False
else:
    try:
        print(f"\nDownloading HIGH RESOLUTION Google Satellite basemap (zoom {target_zoom})...")
        print("  Note: Google tile access may be subject to Terms of Service.")
        print("  For production use, consider Google Earth Engine API.")
        basemap_google_path = download_basemap(
            bbox=bbox,
            output_path=basemap_google_path,
            source="google_satellite",
            zoom=target_zoom
        )
        
        with rasterio.open(basemap_google_path) as src:
            google_hr_width = src.width
            google_hr_height = src.height
            google_hr_bounds = src.bounds
            width_meters = (google_hr_bounds.right - google_hr_bounds.left) * 111320 * math.cos(math.radians(center_lat))
            height_meters = (google_hr_bounds.top - google_hr_bounds.bottom) * 111320
            google_hr_resolution = ((width_meters / google_hr_width) + (height_meters / google_hr_height)) / 2.0
        
        print(f"✓ Google Satellite basemap (HIGH RES) saved: {basemap_google_path}")
        print(f"  Dimensions: {google_hr_width} x {google_hr_height} pixels")
        print(f"  Resolution: {google_hr_resolution:.2f} meters per pixel")
        google_downloaded = True
    except Exception as e:
        print(f"⚠️  Could not download Google Satellite basemap: {e}")
        print("  Continuing with ESRI and OpenStreetMap basemaps only...")
        basemap_google_path = None
        google_downloaded = False
        google_hr_resolution = None

print("\n" + "="*60)
print("Final Basemap Resolution Summary:")
print("="*60)
print(f"ESRI World Imagery (HIGH RES):     {esri_hr_resolution:.2f} m/pixel")
print(f"OpenStreetMap (HIGH RES):          {osm_hr_resolution:.2f} m/pixel")
if google_downloaded and google_hr_resolution:
    print(f"Google Satellite (HIGH RES):      {google_hr_resolution:.2f} m/pixel")
print(f"\n✓ High resolution basemaps are ready for analysis!")
if google_downloaded:
    print(f"\nNote: Google Satellite tiles may be subject to Terms of Service restrictions.")
    print(f"      For production use, consider Google Earth Engine API with authentication.")


Bounding box:
  Latitude: 49.338828 to 49.367100
  Longitude: -124.417904 to -124.382271

Step 2a: Download current basemaps and check resolution

✓ ESRI basemap (low res) already exists: outputs/qualicum_beach_basemap_esri_lowres.tif
  REUSING existing file (skipping download)
  Dimensions: 207 x 253 pixels
  Resolution: 12.46 meters per pixel

✓ OpenStreetMap basemap (low res) already exists: outputs/qualicum_beach_basemap_osm_lowres.tif
  REUSING existing file (skipping download)
  Dimensions: 207 x 253 pixels
  Resolution: 12.46 meters per pixel

Current Basemap Resolution Summary:
ESRI World Imagery:     12.46 m/pixel
OpenStreetMap:          12.46 m/pixel

⚠️  These resolutions are too low for detailed analysis!

Target resolution: 0.5 m/pixel
Calculated zoom level: 19
Expected resolution at zoom 19: 0.19 m/pixel

Step 2b: Download HIGH RESOLUTION basemaps

✓ ESRI basemap (HIGH RES) already exists: outputs/qualicum_beach_basemap_esri.tif
  REUSING existing file (skipping download)

## Step 3: Download Drone Imagery from S3


In [4]:
# Setup paths
input_dir = Path("input")
photos_dir = Path("input/images")

# Download all images from input manifest files
print("Downloading images from S3...")
print("=" * 60)
download_stats = download_all_images_from_input_dir(
    input_dir=input_dir,
    photos_dir=photos_dir,
    skip_existing=True  # Don't re-download if images already exist
)
print("=" * 60)
print("✓ Image download complete")


2025-12-09 20:08:17,362 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Found 12 manifest files
2025-12-09 20:08:17,485 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Processing manifest: input-file_172543.txt
2025-12-09 20:08:17,485 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Bucket: spexi-data-domain-assets-production-ca-central-1
2025-12-09 20:08:17,485 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   S3 prefix: standardized-images/8928d89ac53ffff/172543/
2025-12-09 20:08:17,486 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Total images: 152
2025-12-09 20:08:17,489 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Completed: 0 downloaded, 152 skipped, 0 failed
2025-12-09 20:08:17,493 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Processing manifest: input-file_172547.txt
2025-12-09 20:08:17,493 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Bucket: spexi-data-domain-assets-production-ca-central-1
2025-12-09 20:08:17,493 - qual

2025-12-09 20:08:17,531 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Completed: 0 downloaded, 152 skipped, 0 failed
2025-12-09 20:08:17,534 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Processing manifest: input-file_172568.txt
2025-12-09 20:08:17,534 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Bucket: spexi-data-domain-assets-production-ca-central-1
2025-12-09 20:08:17,535 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   S3 prefix: standardized-images/8928d89ac43ffff/172568/
2025-12-09 20:08:17,535 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Total images: 152
2025-12-09 20:08:17,537 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Completed: 0 downloaded, 152 skipped, 0 failed
2025-12-09 20:08:17,540 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Processing manifest: input-file_172571.txt
2025-12-09 20:08:17,540 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Bucket: spexi-data-domain-assets-production-ca-central-1
2025-

✓ Image download complete


## Step 4: Export GCPs for MetaShape


In [5]:
# Export GCPs to MetaShape XML format (preferred by MetaShape)
# Use 0.005m (5mm) accuracy for very high weight in bundle adjustment
# This matches the gcp_accuracy parameter used in Step 6
gcp_accuracy = 0.005  # 5mm accuracy for very high weight
gcp_xml_path = output_dir / "gcps_metashape.xml"
export_to_metashape_xml(gcps, str(gcp_xml_path), accuracy=gcp_accuracy)
print(f"✓ GCPs exported to XML: {gcp_xml_path}")
print(f"  GCP accuracy set to {gcp_accuracy}m (5mm) for high weight in bundle adjustment")

# Also export CSV for reference
gcp_csv_path = output_dir / "gcps_metashape.csv"
export_to_metashape_csv(gcps, str(gcp_csv_path), accuracy=gcp_accuracy)
print(f"✓ GCPs also exported to CSV: {gcp_csv_path}")
print(f"  GCP accuracy set to {gcp_accuracy}m (5mm) for high weight in bundle adjustment")

# Use XML file for processing (MetaShape's native format)
gcp_file_path = gcp_xml_path


Exported 12 GCPs to MetaShape XML: outputs/gcps_metashape.xml
✓ GCPs exported to XML: outputs/gcps_metashape.xml
  GCP accuracy set to 0.005m (5mm) for high weight in bundle adjustment
Exported 12 GCPs to MetaShape CSV: outputs/gcps_metashape.csv
✓ GCPs also exported to CSV: outputs/gcps_metashape.csv
  GCP accuracy set to 0.005m (5mm) for high weight in bundle adjustment


## Step 5: Process Orthomosaic WITHOUT GCPs


In [6]:
# Process orthomosaic WITHOUT GCPs
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path

if 'output_dir' not in locals():
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)

# Setup paths for processing
intermediate_dir = output_dir / "intermediate"
ortho_output_dir = output_dir / "orthomosaics"

# Check if final output GeoTIFF already exists - if so, skip all processing
ortho_path_no_gcps = ortho_output_dir / "orthomosaic_no_gcps.tif"
if ortho_path_no_gcps.exists():
    print("=" * 60)
    print("Step 5: Process Orthomosaic WITHOUT GCPs")
    print("=" * 60)
    print(f"\n✓ Final output GeoTIFF already exists: {ortho_path_no_gcps.absolute()}")
    file_size_mb = ortho_path_no_gcps.stat().st_size / (1024 * 1024)
    print(f"  Size: {file_size_mb:.2f} MB")
    print("\n⏭️  SKIPPING all processing (no project opening, no recomputation)")
    print("   To force recomputation, delete the file above and re-run this cell.")
    
    # Create minimal stats dictionary for compatibility with downstream cells
    project_path_no_gcps = intermediate_dir / "orthomosaic_no_gcps.psx"
    log_dir = intermediate_dir / "logs"
    log_file_path = log_dir / "orthomosaic_no_gcps_metashape.log" if log_dir.exists() else None
    
    stats_no_gcps = {
        'product_id': 'orthomosaic_no_gcps',
        'use_gcps': False,
        'num_photos': 0,  # Unknown without opening project
        'ortho_path': str(ortho_path_no_gcps),
        'project_path': str(project_path_no_gcps),
        'log_file_path': str(log_file_path) if log_file_path and log_file_path.exists() else None,
        'processing_status': {
            'photos_added': True,
            'photos_matched': True,
            'cameras_aligned': True,
            'depth_maps_built': True,
            'model_built': True,
            'orthomosaic_built': True
        },
        'project_loaded': False,
        'skipped_processing': True
    }
    
    print("\n✓ Using existing orthomosaic file - no processing performed")
    print(f"\n📁 Output File:")
    print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_no_gcps.absolute()}")
    print(f"    Size: {file_size_mb:.2f} MB")
else:
    # Output file doesn't exist - proceed with processing
    if 'process_orthomosaic' not in locals() or 'PhotoMatchQuality' not in locals() or 'DepthMapQuality' not in locals():
        from qualicum_beach_gcp_analysis import (
            process_orthomosaic,
            PhotoMatchQuality,
            DepthMapQuality
        )

    if 'photos_dir' not in locals():
        photos_dir = Path("input/images")

    # Process orthomosaic WITHOUT GCPs
    # Note: clean_intermediate_files=False will reuse existing processing steps
    # Set to True to start fresh and delete previous work
    print("=" * 60)
    print("Processing orthomosaic WITHOUT GCPs...")
    print("=" * 60)

    project_path_no_gcps = intermediate_dir / "orthomosaic_no_gcps.psx"

    stats_no_gcps = process_orthomosaic(
        photos_dir=photos_dir,
        output_path=ortho_output_dir,
        project_path=project_path_no_gcps,
        product_id="orthomosaic_no_gcps",
        clean_intermediate_files=False,  # Reuse existing processing if available
        photo_match_quality=PhotoMatchQuality.MediumQuality,
        depth_map_quality=DepthMapQuality.MediumQuality,
        tiepoint_limit=10000,
        use_gcps=False
    )

    print("\n✓ Orthomosaic processing (without GCPs) complete!")
    print(f"  Number of photos: {stats_no_gcps['num_photos']}")
    print(f"\n📁 Output Files:")
    ortho_path_no_gcps = Path(stats_no_gcps['ortho_path'])
    if ortho_path_no_gcps.exists():
        file_size_mb = ortho_path_no_gcps.stat().st_size / (1024 * 1024)
        print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_no_gcps.absolute()}")
        print(f"    Size: {file_size_mb:.2f} MB")
    else:
        print(f"  ✗ Orthomosaic GeoTIFF NOT FOUND at: {ortho_path_no_gcps.absolute()}")
        print(f"    Expected location: {ortho_path_no_gcps}")
        print(f"    Output directory exists: {ortho_path_no_gcps.parent.exists()}")
    if 'log_file_path' in stats_no_gcps:
        print(f"  📝 Log file: {stats_no_gcps['log_file_path']}")


Step 5: Process Orthomosaic WITHOUT GCPs

✓ Final output GeoTIFF already exists: /Users/mauriciohessflores/Documents/Code/MyCode/research-qualicum_beach_gcp_analysis/outputs/orthomosaics/orthomosaic_no_gcps.tif
  Size: 1890.87 MB

⏭️  SKIPPING all processing (no project opening, no recomputation)
   To force recomputation, delete the file above and re-run this cell.

✓ Using existing orthomosaic file - no processing performed

📁 Output File:
  ✓ Orthomosaic GeoTIFF: /Users/mauriciohessflores/Documents/Code/MyCode/research-qualicum_beach_gcp_analysis/outputs/orthomosaics/orthomosaic_no_gcps.tif
    Size: 1890.87 MB


## Step 6: Process Orthomosaic WITH GCPs


In [ ]:
# Process orthomosaic WITH GCPs
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path

if 'output_dir' not in locals():
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)

# Setup paths for processing
intermediate_dir = output_dir / "intermediate"
ortho_output_dir = output_dir / "orthomosaics"

# Check if final output GeoTIFF already exists - if so, skip all processing
ortho_path_with_gcps = ortho_output_dir / "orthomosaic_with_gcps.tif"
if ortho_path_with_gcps.exists():
    print("=" * 60)
    print("Step 6: Process Orthomosaic WITH GCPs")
    print("=" * 60)
    print(f"\n✓ Final output GeoTIFF already exists: {ortho_path_with_gcps.absolute()}")
    file_size_mb = ortho_path_with_gcps.stat().st_size / (1024 * 1024)
    print(f"  Size: {file_size_mb:.2f} MB")
    print("\n⏭️  SKIPPING all processing (no project opening, no recomputation)")
    print("   To force recomputation, delete the file above and re-run this cell.")
    
    # Create minimal stats dictionary for compatibility with downstream cells
    project_path_with_gcps = intermediate_dir / "orthomosaic_with_gcps.psx"
    log_dir = intermediate_dir / "logs"
    log_file_path = log_dir / "orthomosaic_with_gcps_metashape.log" if log_dir.exists() else None
    
    stats_with_gcps = {
        'product_id': 'orthomosaic_with_gcps',
        'use_gcps': True,
        'num_photos': 0,  # Unknown without opening project
        'num_markers': 0,  # Unknown without opening project
        'ortho_path': str(ortho_path_with_gcps),
        'project_path': str(project_path_with_gcps),
        'log_file_path': str(log_file_path) if log_file_path and log_file_path.exists() else None,
        'processing_status': {
            'photos_added': True,
            'photos_matched': True,
            'cameras_aligned': True,
            'depth_maps_built': True,
            'model_built': True,
            'orthomosaic_built': True
        },
        'project_loaded': False,
        'skipped_processing': True
    }
    
    print("\n✓ Using existing orthomosaic file - no processing performed")
    print(f"\n📁 Output File:")
    print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_with_gcps.absolute()}")
    print(f"    Size: {file_size_mb:.2f} MB")
else:
    # Output file doesn't exist - proceed with processing
    if 'process_orthomosaic' not in locals() or 'PhotoMatchQuality' not in locals() or 'DepthMapQuality' not in locals():
        from qualicum_beach_gcp_analysis import (
            process_orthomosaic,
            PhotoMatchQuality,
            DepthMapQuality
        )

    if 'photos_dir' not in locals():
        photos_dir = Path("input/images")

    # Note: clean_intermediate_files=False will reuse existing processing steps
    # Set to True to start fresh and delete previous work
    print("=" * 60)
    print("Processing orthomosaic WITH GCPs...")
    print("=" * 60)

    project_path_with_gcps = intermediate_dir / "orthomosaic_with_gcps.psx"

    # Use XML file (MetaShape's native format) - defined in Step 4
    gcp_file_for_processing = output_dir / "gcps_metashape.xml"

    stats_with_gcps = process_orthomosaic(
    photos_dir=photos_dir,
    output_path=ortho_output_dir,
    project_path=project_path_with_gcps,
    gcp_file=gcp_file_for_processing,  # Use XML file (MetaShape's native format)
    product_id="orthomosaic_with_gcps",
    clean_intermediate_files=False,  # Reuse existing processing if available
    photo_match_quality=PhotoMatchQuality.MediumQuality,
    depth_map_quality=DepthMapQuality.MediumQuality,
    tiepoint_limit=10000,
    use_gcps=True,
        gcp_accuracy=0.005,  # Very high accuracy (5mm) for very high weight in bundle adjustment
    )

    print("\n✓ Orthomosaic processing (with GCPs) complete!")
    print(f"  Number of photos: {stats_with_gcps['num_photos']}")
    print(f"  Number of markers: {stats_with_gcps.get('num_markers', 0)}")
    print(f"\n📁 Output Files:")
    ortho_path_with_gcps = Path(stats_with_gcps['ortho_path'])
    if ortho_path_with_gcps.exists():
        file_size_mb = ortho_path_with_gcps.stat().st_size / (1024 * 1024)
        print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_with_gcps.absolute()}")
        print(f"    Size: {file_size_mb:.2f} MB")
    else:
        print(f"  ✗ Orthomosaic GeoTIFF NOT FOUND at: {ortho_path_with_gcps.absolute()}")
        print(f"    Expected location: {ortho_path_with_gcps}")
        print(f"    Output directory exists: {ortho_path_with_gcps.parent.exists()}")
    if 'log_file_path' in stats_with_gcps:
        print(f"  📝 Log file: {stats_with_gcps['log_file_path']}")


2025-12-09 20:08:17,580 - qualicum_beach_gcp_analysis.metashape_processor - INFO - 📝 MetaShape verbose output will be saved to: outputs/intermediate/logs/orthomosaic_with_gcps_metashape.log
2025-12-09 20:08:17,580 - qualicum_beach_gcp_analysis.metashape_processor - INFO - 🚀 Creating new Metashape project...
2025-12-09 20:08:17,989 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Processing status:


Processing orthomosaic WITH GCPs...
SaveProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
saved project in 0.00202 sec
LoadProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
loaded project in 0.000404 sec


2025-12-09 20:08:17,992 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Photos added: False
2025-12-09 20:08:17,993 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Photos matched: False
2025-12-09 20:08:17,994 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Cameras aligned: False
2025-12-09 20:08:17,994 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Depth maps built: False
2025-12-09 20:08:17,995 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Model built: False
2025-12-09 20:08:17,996 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Orthomosaic built: False
2025-12-09 20:08:17,997 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Adding photos from: input/images
2025-12-09 20:08:18,006 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Found 1841 images


AddPhotos
SaveProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
saved project in 0.137751 sec
LoadProject: path = outputs/intermediate/orthomosaic_with_gcps.psx


2025-12-09 20:08:19,849 - qualicum_beach_gcp_analysis.metashape_processor - INFO - ============================================================
2025-12-09 20:08:19,850 - qualicum_beach_gcp_analysis.metashape_processor - INFO - GROUND CONTROL POINTS (GCPs) CONFIGURATION
2025-12-09 20:08:19,850 - qualicum_beach_gcp_analysis.metashape_processor - INFO - ============================================================
2025-12-09 20:08:19,851 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Loading GCPs from file: outputs/gcps_metashape.xml
2025-12-09 20:08:19,851 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Detected XML format, attempting importMarkers


...

OptimizeCameras: f, cx, cy, k1-k3, p1, p2
adjusting: xxxxxxxx 0.172261 -> 0.172239
coordinates applied in 0 sec
SaveProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
saved project in 7.1e-05 sec
LoadProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
loaded project in 0.105437 sec


2025-12-09 20:21:19,562 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   ✓ Camera optimization complete
2025-12-09 20:21:19,564 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Building depth maps...


BuildDepthMaps: quality = Medium, depth filtering = Mild, PM version
Preparing 1799 cameras info...
cameras data loaded in 0.054794 s
cameras graph built in 2.15857 s
filtering neighbors with too low common points, threshold=50...
Camera 251 has no neighbors
Camera 257 has no neighbors
Camera 1449 has no neighbors
avg neighbors before -> after filtering: 45.9844 -> 17.7604 (61% filtered out)
limiting neighbors to 16 best...
avg neighbors before -> after filtering: 17.7604 -> 14.3046 (19% filtered out)
neighbors number min/1%/10%/median/90%/99%/max: 0, 2, 9, median=16, 16, 16, 16
cameras info prepared in 4.03799 s
saved cameras info in 0.017256
Partitioning 1799 cameras...
number of mini clusters: 37
37 groups: avg_ref=48.6216 avg_neighb=45.6757 total_io=194%
max_ref=50 max_neighb=77 max_total=124
cameras partitioned in 0.015145 s
saved depth map partition in 0.0007 sec
loaded cameras info in 0.035659
loaded depth map partition in 0.000112 sec
already partitioned (50<=50 ref cameras, 27

2025-12-09 20:43:37,372 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   ✓ Depth maps built
2025-12-09 20:43:37,374 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Building 3D model...
2025-12-09 20:43:37,375 - qualicum_beach_gcp_analysis.metashape_processor - INFO -   Verifying image file accessibility...


SaveProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
saved project in 7.2e-05 sec
LoadProject: path = outputs/intermediate/orthomosaic_with_gcps.psx
loaded project in 0.109476 sec
BuildModel: source data = Depth maps, surface type = Arbitrary, face count = High, volumetric masking = 0, OOC version, interpolation = Enabled, vertex colors = 1
Compression level: 1
Preparing depth maps...
1796 depth maps
scheduled 90 depth map groups (1796 cameras)
saved camera partition in 0.001492 sec
loaded camera partition in 0.000521 sec
saved group #1/90: done in 0.980627 s, 20 cameras, 62.2326 MB data, 24.1406 KB registry
loaded camera partition in 0.000186 sec
saved group #2/90: done in 1.01369 s, 20 cameras, 64.9307 MB data, 24.1406 KB registry
loaded camera partition in 0.000228 sec
saved group #3/90: done in 1.19501 s, 20 cameras, 62.0259 MB data, 24.1406 KB registry
loaded camera partition in 0.000218 sec
saved group #4/90: done in 1.12336 s, 20 cameras, 63.6548 MB data, 24.1406 K

## Step 7: Compare No-GCP Orthomosaic to Reference Basemaps

### Comparison Methodology

The no-GCP orthomosaic is compared to reference basemaps (ESRI World Imagery and Google Satellite) using a comprehensive methodology:

#### 1. **Reprojection and Alignment**
   - The orthomosaic is reprojected to match the reference basemap's coordinate reference system (CRS) and spatial extent
   - Bilinear resampling is used to ensure pixel-level alignment
   - Both rasters are normalized to the same spatial resolution

#### 2. **Pixel-level Error Metrics**
   - **RMSE (Root Mean Square Error)**: Measures overall pixel intensity differences between orthomosaic and reference
   - **MAE (Mean Absolute Error)**: Measures average absolute pixel differences
   - **Structural Similarity**: Correlation-based measure indicating how well the orthomosaic structure matches the reference

#### 3. **2D Spatial Error (Feature Matching)**
   - Feature-based matching (SIFT, ORB, or phase correlation) identifies corresponding points between orthomosaic and reference
   - Computes X and Y pixel offsets, providing 2D spatial error measurements
   - This identifies systematic shifts, rotations, or misalignments that pixel-level metrics might miss
   - The method automatically selects the best available algorithm (SIFT > ORB > phase correlation > template matching)

#### 4. **Seamline Detection**
   - Gradient-based analysis detects potential seamline artifacts
   - Identifies high-gradient regions that may indicate stitching errors or discontinuities
   - Reports percentage of pixels flagged as potential seamlines

#### 5. **Comparison Process**
   - The no-GCP orthomosaic (with and without GCPs) are compared against the same reference basemap
   - Metrics are computed for each band and averaged for overall statistics
   - Results are saved to JSON files for persistence and later analysis


## Step 7: Compare No-GCP Orthomosaic to Reference Basemaps


In [1]:
# Compare against HIGH RESOLUTION basemaps (ESRI and Google)
# NOTE: Only processing "no GCPs" orthomosaic since GCPs are not being detected in images
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path
import rasterio

# NOTE: Feature matching uses AROSICS (if available) for robust satellite image co-registration
#       AROSICS is better suited for satellite imagery than ORB/SIFT
#       Install with: conda install -c conda-forge arosics>=1.3.0

if 'compare_orthomosaic_to_basemap' not in locals() or 'downsample_ortho_to_basemap_resolution' not in locals():
    from qualicum_beach_gcp_analysis import (
# NOTE: Feature matching uses AROSICS (if available) for robust satellite image co-registration
#       AROSICS is better suited for satellite imagery than ORB/SIFT
#       Install with: conda install -c conda-forge arosics>=1.3.0

        compare_orthomosaic_to_basemap,
        downsample_ortho_to_basemap_resolution
    )

if 'output_dir' not in locals():
    output_dir = Path("outputs")

print("=" * 60)
print("Step 7: Compare No-GCP Orthomosaic to HIGH RESOLUTION Basemaps")
print("=" * 60)
print("\nUsing HIGH RESOLUTION basemaps only (ESRI and Google Satellite)")
print("Feature matching uses ORB only, with orthomosaic downsampled to match basemap resolution")
print("\nNOTE: Only processing 'no GCPs' orthomosaic since GCP markers are not being detected in images")

comparison_dir = output_dir / "comparisons"
comparison_dir.mkdir(parents=True, exist_ok=True)

# Ensure HIGH RESOLUTION basemap paths are defined (from Step 2)
if 'basemap_esri_path' not in locals():
    basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri_utm10n.tif")if 'basemap_google_path' not in locals():
    basemap_google_path = str(output_dir / "qualicum_beach_basemap_google_utm10n.tif")
# Verify basemap resolutions
# Helper function to calculate resolution in meters from WGS84 transform
def get_resolution_meters(src):
    """Calculate resolution in meters per pixel from a rasterio dataset."""
    import math
    transform = src.transform
    crs = src.crs
    
    # If CRS is WGS84 (EPSG:4326), transform values are in degrees
    if crs and crs.to_string() == 'EPSG:4326':
        # Get center latitude for conversion
        bounds = src.bounds
        center_lat = (bounds.bottom + bounds.top) / 2.0
        
        # Convert degrees to meters
        # Longitude: varies with latitude
        meters_per_degree_lon = 111320.0 * math.cos(math.radians(center_lat))
        # Latitude: constant
        meters_per_degree_lat = 111320.0
        
        # Transform values are in degrees
        res_x_deg = abs(transform[0])
        res_y_deg = abs(transform[4])
        
        # Convert to meters
        res_x_m = res_x_deg * meters_per_degree_lon
        res_y_m = res_y_deg * meters_per_degree_lat
        resolution = (res_x_m + res_y_m) / 2.0
    else:
        # For projected CRS, transform values are already in meters
        res_x = abs(transform[0])
        res_y = abs(transform[4])
        resolution = (res_x + res_y) / 2.0
    
    return resolution

print("\nBasemap resolutions:")
basemap_resolution = None
for name, path in [("ESRI", basemap_esri_path), ("Google", basemap_google_path)]:
    if Path(path).exists():
        with rasterio.open(path) as src:
            resolution = get_resolution_meters(src)
            if basemap_resolution is None:
                basemap_resolution = resolution
            print(f"  {name}: {resolution:.4f} m/pixel")
    else:
        print(f"  {name}: NOT FOUND at {path}")

# Determine orthomosaic path - only use "no GCPs" version
if 'stats_no_gcps' in locals() and 'ortho_path' in stats_no_gcps:
    ortho_no_gcps_path = Path(stats_no_gcps['ortho_path'])
else:
    # Try to find the orthomosaic file directly
    ortho_output_dir = output_dir / "orthomosaics"
    ortho_no_gcps_path = ortho_output_dir / "orthomosaic_no_gcps_utm10n.tif"    if not ortho_no_gcps_path.exists():
        raise FileNotFoundError(f"Orthomosaic (no GCPs) not found at: {ortho_no_gcps_path.absolute()}\n"
                               f"Please run Step 5 first, or ensure the file exists at this location.")

print(f"\nUsing orthomosaic (no GCPs): {ortho_no_gcps_path}")

# Get orthomosaic resolution for comparison
with rasterio.open(ortho_no_gcps_path) as src:
    ortho_resolution = get_resolution_meters(src)
    print(f"\nOrthomosaic resolution: {ortho_resolution:.4f} m/pixel")
    if basemap_resolution:
        print(f"Basemap resolution: {basemap_resolution:.4f} m/pixel")
        downsample_factor = ortho_resolution / basemap_resolution
        print(f"Downsampling factor: {downsample_factor:.2f}x")
        print(f"Creating downsampled version of orthomosaic for feature matching at same resolution...")

# Create downsampled version of ortho for feature matching
# Use ESRI basemap as reference for downsampling (all basemaps should have same resolution)
downsampled_dir = comparison_dir / "downsampled_orthos"
downsampled_dir.mkdir(parents=True, exist_ok=True)

ortho_no_gcps_downsampled = downsampled_dir / "orthomosaic_no_gcps_downsampled.tif"

# Check if downsampled version already exists
if not ortho_no_gcps_downsampled.exists():
    print(f"\nCreating downsampled version of orthomosaic (no GCPs)...")
    downsample_ortho_to_basemap_resolution(
        ortho_path=ortho_no_gcps_path,
        basemap_path=Path(basemap_esri_path),
        output_path=ortho_no_gcps_downsampled
    )
else:
    print(f"\n✓ Downsampled orthomosaic (no GCPs) already exists: {ortho_no_gcps_downsampled}")

print("\n✓ Downsampled orthomosaic ready for feature matching")

# Compare against ESRI basemap
print("\n" + "=" * 60)
print("Comparing to ESRI World Imagery basemap...")
print("=" * 60)

print("\nComparing orthomosaic (no GCPs) to ESRI basemap...")
print("  Using downsampled orthomosaic and AROSICS feature matching (with ORB fallback)...")
# NOTE: Feature matching uses AROSICS (if available) for robust satellite image co-registration
#       AROSICS is better suited for satellite imagery than ORB/SIFT
#       Install with: conda install -c conda-forge arosics>=1.3.0

metrics_no_gcps_esri = compare_orthomosaic_to_basemap(
    ortho_path=ortho_no_gcps_downsampled,  # Use downsampled version
    basemap_path=Path(basemap_esri_path),
    output_dir=comparison_dir,
    feature_matching_method='arosics'  # Use AROSICS (falls back to ORB if not available)
)

print("\n✓ ESRI comparison complete!")

print("\n" + "=" * 60)

print("  Using downsampled orthomosaic and AROSICS feature matching (with ORB fallback)...")

# Compare against Google Satellite basemap (if available)
if Path(basemap_google_path).exists():
    print("\n" + "=" * 60)
    print("Comparing to Google Satellite basemap...")
    print("=" * 60)
    
    print("\nComparing orthomosaic (no GCPs) to Google Satellite basemap...")
    print("  Using downsampled orthomosaic and AROSICS feature matching (with ORB fallback)...")
# NOTE: Feature matching uses AROSICS (if available) for robust satellite image co-registration
#       AROSICS is better suited for satellite imagery than ORB/SIFT
#       Install with: conda install -c conda-forge arosics>=1.3.0

    metrics_no_gcps_google = compare_orthomosaic_to_basemap(
        ortho_path=ortho_no_gcps_downsampled,  # Use downsampled version
        basemap_path=Path(basemap_google_path),
        output_dir=comparison_dir,
        feature_matching_method='arosics'  # Use AROSICS (falls back to ORB if not available)
    )
    
    print("\n✓ Google Satellite comparison complete!")
else:
    print("\n⚠️  Google Satellite basemap not found. Skipping Google comparison.")
    metrics_no_gcps_google = None

# Save metrics to JSON files for later use
import json
from qualicum_beach_gcp_analysis.report_generator import convert_to_json_serializable

metrics_dir = comparison_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

# Save all metrics (only no GCPs)
from qualicum_beach_gcp_analysis.report_generator import convert_to_json_serializable

# ESRI metrics
metrics_no_gcps_esri_serializable = convert_to_json_serializable(metrics_no_gcps_esri)
with open(metrics_dir / "metrics_no_gcps_esri.json", 'w') as f:
    json.dump(metrics_no_gcps_esri_serializable, f, indent=2)

# Google metrics (if available)
if metrics_no_gcps_google is not None:
    metrics_no_gcps_google_serializable = convert_to_json_serializable(metrics_no_gcps_google)
    with open(metrics_dir / "metrics_no_gcps_google.json", 'w') as f:
        json.dump(metrics_no_gcps_google_serializable, f, indent=2)

print(f"\n✓ All metrics saved to: {metrics_dir}")



Step 7: Compare No-GCP Orthomosaic to HIGH RESOLUTION Basemaps

Using HIGH RESOLUTION basemaps only (ESRI and Google Satellite)
Feature matching uses ORB only, with orthomosaic downsampled to match basemap resolution

NOTE: Only processing 'no GCPs' orthomosaic since GCP markers are not being detected in images

Basemap resolutions:
  ESRI: 0.1945 m/pixel
  Google: 0.1945 m/pixel

Using orthomosaic (no GCPs): outputs/orthomosaics/orthomosaic_no_gcps.tif

Orthomosaic resolution: 0.0293 m/pixel
Basemap resolution: 0.1945 m/pixel
Downsampling factor: 0.15x
Creating downsampled version of orthomosaic for feature matching at same resolution...

✓ Downsampled orthomosaic (no GCPs) already exists: outputs/comparisons/downsampled_orthos/orthomosaic_no_gcps_downsampled.tif

✓ Downsampled orthomosaic ready for feature matching

Comparing to ESRI World Imagery basemap...

Comparing orthomosaic (no GCPs) to ESRI basemap...
  Using downsampled orthomosaic and AROSICS feature matching (with ORB fall

/Users/mauriciohessflores/Library/Python/3.9/lib/python/site-packages/arosics/CoReg.py:122: UserWarning: The footprint of the reference image contains multiple separate image parts. AROSICS will only process the largest image part.
  warnings.warn(f'The footprint of the {self.imName} contains multiple separate image parts. '


Bounding box of calculated footprint for reference image:
	(-124.41111085905358, 49.34707968036169, -124.38911761867956, 49.35930232588659)


/Users/mauriciohessflores/Library/Python/3.9/lib/python/site-packages/geoarray/baseclasses.py:806: UserWarning: 
Automatic nodata value detection returned the value 204.0 for GeoArray 'qualicum_beach_basemap_esri' but this seems to be unreliable (occurs in only 1 image corner). To avoid automatic detection, just pass the correct nodata value.
  warnings.warn("\nAutomatic nodata value detection returned the value %s for GeoArray '%s' but this "


Automatically detected nodata value for GeoArray_CoReg 'qualicum_beach_basemap_esri': 204.0
Calculating footprint polygon and actual data corner coordinates for image to be shifted...


...


✓ ESRI comparison complete!

  Using downsampled orthomosaic and AROSICS feature matching (with ORB fallback)...

Comparing to Google Satellite basemap...

Comparing orthomosaic (no GCPs) to Google Satellite basemap...
  Using downsampled orthomosaic and AROSICS feature matching (with ORB fallback)...
Automatically detected nodata value for GeoArray_CoReg 'reprojected_orthomosaic_no_gcps_downsampled': 0.0
Calculating footprint polygon and actual data corner coordinates for reference image...


/Users/mauriciohessflores/Library/Python/3.9/lib/python/site-packages/arosics/CoReg.py:122: UserWarning: The footprint of the reference image contains multiple separate image parts. AROSICS will only process the largest image part.
  warnings.warn(f'The footprint of the {self.imName} contains multiple separate image parts. '


Bounding box of calculated footprint for reference image:
	(-124.41111085905358, 49.34707968036169, -124.38911761867956, 49.35930232588659)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(-124.41790447957143, 49.338827689428626, -124.38227074599993, 49.36709958600006)
Matching window position (X,Y): -124.40011424015476/49.35319260342664
Detected integer shifts (X/Y):                            -3/-8
Detected subpixel shifts (X/Y):                           0.3381528854370117/0.30063435435295105
Calculated total shifts in fft pixel units (X/Y):         -2.6618471145629883/-7.699365645647049
Calculated total shifts in reference pixel units (X/Y):   -2.6618471145629883/-7.699365645647049
Calculated total shifts in target pixel units (X/Y):      -2.6618471145629883/-7.699365645647049
Calculated map shifts (X,Y):                              -7.139747907558558e-06/1.345254738538415e-0

Feature matching (orb) failed: OpenCV(4.12.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/features2d/src/matchers.cpp:860: error: (-215:Assertion failed) trainDescCollection[iIdx].rows < IMGIDX_ONE in function 'knnMatchImpl'

Feature matching (sift) failed: OpenCV(4.12.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/features2d/src/matchers.cpp:860: error: (-215:Assertion failed) trainDescCollection[iIdx].rows < IMGIDX_ONE in function 'knnMatchImpl'

Phase correlation failed: local variable 'phase_cross_correlation' referenced before assignment



✓ Google Satellite comparison complete!

✓ All metrics saved to: outputs/comparisons/metrics


## Step 7.5: Apply 2D Shift Alignment and Re-analyze

After the initial comparison, we apply 2D shifts to align the no-GCP orthomosaic with the ESRI basemap using feature matching, then re-analyze accuracy and seamlines to see if alignment improves the results.


In [ ]:
# Step 7.5: Apply Comprehensive Transformation Alignment to No-GCP Orthomosaic and Re-analyze
# NOTE: Only processing "no GCPs" orthomosaic since GCP markers are not being detected in images
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path
if 'json' not in locals():
    import json
import rasterio

if 'apply_comprehensive_transformation_to_orthomosaic' not in locals() or 'compare_orthomosaic_to_basemap' not in locals():
    from qualicum_beach_gcp_analysis import (
        apply_comprehensive_transformation_to_orthomosaic,
        compare_orthomosaic_to_basemap
    )

if 'output_dir' not in locals():
    output_dir = Path("outputs")
if 'comparison_dir' not in locals():
    comparison_dir = output_dir / "comparisons"
if 'basemap_esri_path' not in locals():
    basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri_utm10n.tif")
# Ensure original orthomosaic path is defined (for creating transformed version)
if 'ortho_no_gcps_path' not in locals():
    ortho_output_dir = output_dir / "orthomosaics"
    ortho_no_gcps_path = ortho_output_dir / "orthomosaic_no_gcps_utm10n.tif"
print("=" * 60)
print("Step 7.5: Apply Comprehensive Transformation Alignment to No-GCP Orthomosaic and Re-analyze")
print("=" * 60)
print("Computing multiple transformation types (shift, affine, homography, deformable)")
print("Choosing best transformation based on RMSE with RANSAC outlier removal")
print("Using ORB-only feature matching")
print("Errors will be reported in both pixels and meters")

# Get pixel resolution from basemap
with rasterio.open(basemap_esri_path) as ref:
    pixel_resolution = abs(ref.transform[0])  # meters per pixel
    print(f"
Basemap pixel resolution: {pixel_resolution:.4f} m/pixel")

# Create directory for transformed orthomosaic
shifted_dir = output_dir / "orthomosaics_shifted"
shifted_dir.mkdir(parents=True, exist_ok=True)

# Setup log directory
log_dir = comparison_dir / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Apply comprehensive transformation to orthomosaic (no GCPs)
print("
1. Applying comprehensive transformation to orthomosaic (no GCPs)...")
print("   Computing shift, affine, homography, and deformable transformations...")
shifted_no_gcps_path = shifted_dir / "orthomosaic_no_gcps_transformed.tif"
log_file_no_gcps = log_dir / "transformation_no_gcps.log"
_, trans_info_no_gcps = apply_comprehensive_transformation_to_orthomosaic(
    ortho_path=Path(ortho_no_gcps_path),  # Use original full-resolution ortho
    reference_path=Path(basemap_esri_path),
    output_path=shifted_no_gcps_path,
    feature_matching_method='arosics',  # ORB only
    pixel_resolution=pixel_resolution,
    log_file_path=log_file_no_gcps,
    create_visualization=True
)
print(f"   Best transformation: {trans_info_no_gcps['transformation_type']} (RMSE: {trans_info_no_gcps['rmse_pixels']:.2f} pixels)")
if trans_info_no_gcps['transformation_type'] == 'shift':
    print(f"   Shift applied: X={trans_info_no_gcps['shift_x_pixels']:.2f} px, Y={trans_info_no_gcps['shift_y_pixels']:.2f} px")
    if 'shift_x_meters' in trans_info_no_gcps:
        print(f"   In meters: X={trans_info_no_gcps['shift_x_meters']:.4f} m, Y={trans_info_no_gcps['shift_y_meters']:.4f} m")
if 'secondary_type' in trans_info_no_gcps:
    print(f"   Second best: {trans_info_no_gcps['secondary_type']} (RMSE: {trans_info_no_gcps['secondary_rmse']:.2f} pixels)")
print(f"   Log file: {log_file_no_gcps}")

# Re-analyze transformed orthomosaic (comprehensive analysis including seamlines)
print("
2. Re-analyzing transformed orthomosaic against ESRI basemap...")
print("   This includes RMSE, MAE, seamline detection, and 2D spatial error analysis...")

# Compare shifted orthomosaic (no GCPs)
print("
   Analyzing shifted orthomosaic (no GCPs)...")
print("   Using ORB-only feature matching...")
metrics_shifted_no_gcps = compare_orthomosaic_to_basemap(
    ortho_path=shifted_no_gcps_path,
    basemap_path=Path(basemap_esri_path),
    output_dir=comparison_dir,
    feature_matching_method='arosics'  # ORB only
)

# Output errors in meters
if metrics_shifted_no_gcps.get('overall', {}).get('errors_2d', {}).get('rmse_2d_pixels'):
    rmse_2d_px = metrics_shifted_no_gcps['overall']['errors_2d']['rmse_2d_pixels']
    rmse_2d_m = rmse_2d_px * pixel_resolution
    print(f"   2D RMSE: {rmse_2d_px:.2f} pixels = {rmse_2d_m:.4f} meters")

# Save shifted metrics
metrics_dir = comparison_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

shifted_metrics_file_no_gcps = metrics_dir / "metrics_shifted_no_gcps_esri.json"

with open(shifted_metrics_file_no_gcps, 'w') as f:
    json.dump(metrics_shifted_no_gcps, f, indent=2, default=str)

print(f"
✓ Shifted metrics saved to: {metrics_dir}")

# Compare initial vs shifted results
print("
3. Comparing initial vs. shifted results...")

# Load initial metrics if not available
if 'metrics_no_gcps_esri' not in locals():
    metrics_file_no_gcps = metrics_dir / "metrics_no_gcps_esri.json"
    if metrics_file_no_gcps.exists():
        with open(metrics_file_no_gcps, 'r') as f:
            metrics_no_gcps_esri = json.load(f)
    else:
        raise NameError("Initial metrics not found. Please run Step 7 first.")

# Calculate improvements from shifting
initial_no_gcps = metrics_no_gcps_esri.get('overall', {})
shifted_no_gcps = metrics_shifted_no_gcps.get('overall', {})

print("Results:")
if initial_no_gcps.get('rmse') and shifted_no_gcps.get('rmse'):
    rmse_improvement = ((initial_no_gcps['rmse'] - shifted_no_gcps['rmse']) / initial_no_gcps['rmse']) * 100
    print(f"     RMSE: {initial_no_gcps['rmse']:.4f} → {shifted_no_gcps['rmse']:.4f} ({rmse_improvement:+.2f}%)")
if initial_no_gcps.get('seamline_percentage') and shifted_no_gcps.get('seamline_percentage'):
    seamline_improvement = initial_no_gcps['seamline_percentage'] - shifted_no_gcps['seamline_percentage']
    print(f"     Seamlines: {initial_no_gcps['seamline_percentage']:.2f}% → {shifted_no_gcps['seamline_percentage']:.2f}% ({seamline_improvement:+.2f}%)")

print("✓ Step 7.5 complete!")


## Step 7.6: Align No-GCP Orthomosaic to Ground Control Points and Re-analyze

In addition to feature-matching alignment, we also align the no-GCP orthomosaic directly to the ground control points (GCPs) using their known coordinates. This provides an alternative alignment method that can be compared against feature-matching alignment.


In [ ]:
# Step 7.6: Align No-GCP Orthomosaic to Ground Control Points and Re-analyze
# NOTE: Only processing "no GCPs" orthomosaic since GCP markers are not being detected in images
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path
if 'json' not in locals():
    import json

if 'align_orthomosaic_to_gcps' not in locals() or 'compare_orthomosaic_to_basemap' not in locals():
    from qualicum_beach_gcp_analysis import (
        align_orthomosaic_to_gcps,
        compare_orthomosaic_to_basemap
    )

if 'load_gcps_from_kmz' not in locals():
    from qualicum_beach_gcp_analysis import load_gcps_from_kmz

if 'output_dir' not in locals():
    output_dir = Path("outputs")
if 'comparison_dir' not in locals():
    comparison_dir = output_dir / "comparisons"
if 'basemap_esri_path' not in locals():
    basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri_utm10n.tif")
# Ensure orthomosaic path is defined
if 'ortho_no_gcps_path' not in locals():
    ortho_output_dir = output_dir / "orthomosaics"
    ortho_no_gcps_path = ortho_output_dir / "orthomosaic_no_gcps_utm10n.tif"
# Load GCPs if not already loaded
if 'gcps' not in locals():
    # Try to find KMZ file
    kmz_path = Path("input") / "QualicumBeach_AOI.kmz"
    if not kmz_path.exists():
        # Try alternative location
        kmz_path = Path("/Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz")
    
    if kmz_path.exists():
        gcps = load_gcps_from_kmz(str(kmz_path))
        print(f"Loaded {len(gcps)} GCPs from {kmz_path}")
    else:
        raise FileNotFoundError(f"Could not find GCP KMZ file. Tried: {kmz_path}")

print("=" * 60)
print("Step 7.6: Align No-GCP Orthomosaic to Ground Control Points")
print("=" * 60)

# Create directory for GCP-aligned orthomosaic
gcp_aligned_dir = output_dir / "orthomosaics_gcp_aligned"
gcp_aligned_dir.mkdir(parents=True, exist_ok=True)

# Align orthomosaic (no GCPs) to GCPs
print("\n1. Aligning orthomosaic (no GCPs) to GCPs...")
gcp_aligned_no_gcps_path = gcp_aligned_dir / "orthomosaic_no_gcps_gcp_aligned.tif"
_, alignment_info_no_gcps = align_orthomosaic_to_gcps(
    ortho_path=Path(ortho_no_gcps_path),
    reference_path=Path(basemap_esri_path),
    gcps=gcps,
    output_path=gcp_aligned_no_gcps_path
)
print(f"   Alignment RMSE: {alignment_info_no_gcps['rmse_total_pixels']:.2f} pixels")
print(f"   Used {alignment_info_no_gcps['num_gcps_used']} GCPs")

# Re-analyze GCP-aligned orthomosaic
print("\n2. Re-analyzing GCP-aligned orthomosaic against ESRI basemap...")

# Compare GCP-aligned orthomosaic (no GCPs)
print("\n   Analyzing GCP-aligned orthomosaic (no GCPs)...")
metrics_gcp_aligned_no_gcps = compare_orthomosaic_to_basemap(
    ortho_path=gcp_aligned_no_gcps_path,
    basemap_path=Path(basemap_esri_path),
    output_dir=comparison_dir
)

# Save GCP-aligned metrics
metrics_dir = comparison_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

gcp_aligned_metrics_file_no_gcps = metrics_dir / "metrics_gcp_aligned_no_gcps_esri.json"

with open(gcp_aligned_metrics_file_no_gcps, 'w') as f:
    json.dump(metrics_gcp_aligned_no_gcps, f, indent=2, default=str)

print(f"\n✓ GCP-aligned metrics saved to: {metrics_dir}")

# Compare alignment methods
print("\n3. Comparing alignment methods...")

# Load initial and shifted metrics if available
if 'metrics_no_gcps_esri' not in locals():
    metrics_file_no_gcps = metrics_dir / "metrics_no_gcps_esri.json"
    if metrics_file_no_gcps.exists():
        with open(metrics_file_no_gcps, 'r') as f:
            metrics_no_gcps_esri = json.load(f)

# Load shifted metrics if available
shifted_metrics_file_no_gcps = metrics_dir / "metrics_shifted_no_gcps_esri.json"
metrics_shifted_no_gcps = None
if shifted_metrics_file_no_gcps.exists():
    with open(shifted_metrics_file_no_gcps, 'r') as f:
        metrics_shifted_no_gcps = json.load(f)

print("\n   RMSE comparison:")
initial_no = metrics_no_gcps_esri.get('overall', {}) if 'metrics_no_gcps_esri' in locals() else {}
shifted_no = metrics_shifted_no_gcps.get('overall', {}) if metrics_shifted_no_gcps else {}
gcp_aligned_no = metrics_gcp_aligned_no_gcps.get('overall', {})

if initial_no.get('rmse'):
    print(f"     Initial:        {initial_no['rmse']:.4f}")
if shifted_no.get('rmse'):
    print(f"     Feature-matched: {shifted_no['rmse']:.4f}")
if gcp_aligned_no.get('rmse'):
    print(f"     GCP-aligned:     {gcp_aligned_no['rmse']:.4f}")

print("\n✓ Step 7.6 complete!")

In [ ]:
# Compare against OpenStreetMap basemap
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path

if 'compare_orthomosaic_to_basemap' not in locals():
    from qualicum_beach_gcp_analysis import compare_orthomosaic_to_basemap

if 'output_dir' not in locals():
    output_dir = Path("outputs")
if 'comparison_dir' not in locals():
    comparison_dir = output_dir / "comparisons"
if 'basemap_osm_path' not in locals():
    basemap_osm_path = str(output_dir / "qualicum_beach_basemap_osm.tif")

print("=" * 60)
print("Comparing orthomosaics to OpenStreetMap basemap...")
print("=" * 60)

# Determine orthomosaic paths - use stats if available, otherwise find files directly
if 'ortho_no_gcps_path' not in locals():
    if 'stats_no_gcps' in locals() and 'ortho_path' in stats_no_gcps:
        ortho_no_gcps_path = Path(stats_no_gcps['ortho_path'])
    else:
        ortho_output_dir = output_dir / "orthomosaics"
        ortho_no_gcps_path = ortho_output_dir / "orthomosaic_no_gcps.tif"
        if not ortho_no_gcps_path.exists():
            raise FileNotFoundError(f"Orthomosaic (no GCPs) not found at: {ortho_no_gcps_path.absolute()}")

if 'ortho_with_gcps_path' not in locals():
    if 'stats_with_gcps' in locals() and 'ortho_path' in stats_with_gcps:
        ortho_with_gcps_path = Path(stats_with_gcps['ortho_path'])
    else:
        ortho_output_dir = output_dir / "orthomosaics"
        ortho_with_gcps_path = ortho_output_dir / "orthomosaic_with_gcps.tif"
        if not ortho_with_gcps_path.exists():
            raise FileNotFoundError(f"Orthomosaic (with GCPs) not found at: {ortho_with_gcps_path.absolute()}")

# Compare without GCPs
print("\nComparing orthomosaic (without GCPs) to OpenStreetMap basemap...")
metrics_no_gcps_osm = compare_orthomosaic_to_basemap(
    ortho_path=ortho_no_gcps_path,
    basemap_path=Path(basemap_osm_path),
    output_dir=comparison_dir
)

# Compare with GCPs
print("\nComparing orthomosaic (with GCPs) to OpenStreetMap basemap...")
metrics_with_gcps_osm = compare_orthomosaic_to_basemap(
    ortho_path=ortho_with_gcps_path,
    basemap_path=Path(basemap_osm_path),
    output_dir=comparison_dir
)

print("\n✓ OpenStreetMap comparison complete!")

# Save metrics to JSON files for later use
if 'json' not in locals():
    import json
if 'convert_to_json_serializable' not in locals():
    from qualicum_beach_gcp_analysis.report_generator import convert_to_json_serializable

if 'metrics_dir' not in locals():
    metrics_dir = comparison_dir / "metrics"
    metrics_dir.mkdir(parents=True, exist_ok=True)

# Save OpenStreetMap metrics
metrics_no_gcps_osm_serializable = convert_to_json_serializable(metrics_no_gcps_osm)
metrics_with_gcps_osm_serializable = convert_to_json_serializable(metrics_with_gcps_osm)

with open(metrics_dir / "metrics_no_gcps_osm.json", 'w') as f:
    json.dump(metrics_no_gcps_osm_serializable, f, indent=2)
with open(metrics_dir / "metrics_with_gcps_osm.json", 'w') as f:
    json.dump(metrics_with_gcps_osm_serializable, f, indent=2)

print(f"✓ Metrics saved to: {metrics_dir}")


## Step 8: Generate Quality Reports


In [ ]:
# Generate report for ESRI comparison (NO GCPs only)
# NOTE: Only processing "no GCPs" orthomosaic since GCP markers are not being detected in images
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path

# Import report generation functions
if 'generate_comparison_report' not in locals() or 'generate_markdown_report' not in locals() or 'generate_latex_report' not in locals():
    from qualicum_beach_gcp_analysis import (
        generate_comparison_report,
        generate_markdown_report,
        generate_latex_report
    )

# Import visualization functions separately to ensure they're always available
if 'create_error_visualization' not in locals() or 'create_seamline_visualization' not in locals():
    from qualicum_beach_gcp_analysis import (
        create_error_visualization,
        create_seamline_visualization
    )
    
if 'numpy' not in locals():
    import numpy as np
if 'rasterio' not in locals():
    import rasterio
if 'json' not in locals():
    import json

if 'output_dir' not in locals():
    output_dir = Path("outputs")
if 'comparison_dir' not in locals():
    comparison_dir = output_dir / "comparisons"

# Load metrics from Step 7 if available, otherwise check for saved files
if 'metrics_no_gcps_esri' not in locals():
    # Try to load from saved JSON files
    metrics_dir = comparison_dir / "metrics"
    metrics_file_no_gcps = metrics_dir / "metrics_no_gcps_esri.json"
    
    if metrics_file_no_gcps.exists():
        print(f"Loading saved metrics from: {metrics_dir}")
        with open(metrics_file_no_gcps, 'r') as f:
            metrics_no_gcps_esri = json.load(f)
        print("✓ Loaded saved ESRI comparison metrics")
    else:
        raise NameError(
            f"metrics_no_gcps_esri must be defined. "
            f"Please run Step 7 first, or ensure saved metrics exist at:
"
            f"  {metrics_file_no_gcps}"
        )

print("=" * 60)
print("Generating quality reports (NO GCPs only)...")
print("=" * 60)

# ESRI report - generate with only no_gcps metrics
report_json_esri = output_dir / "quality_report_esri.json"
report_md_esri = output_dir / "quality_report_esri.md"

generate_comparison_report(
    metrics_with_gcps=None,  # Not using GCPs
    metrics_without_gcps=metrics_no_gcps_esri,
    output_path=report_json_esri,
    basemap_source="ESRI World Imagery"
)

generate_markdown_report(
    json_report_path=report_json_esri,
    output_path=report_md_esri
)

print(f"
✓ ESRI report saved:")
print(f"  JSON: {report_json_esri}")
print(f"  Markdown: {report_md_esri}")

# Ensure orthomosaic path is defined
if 'ortho_no_gcps_path' not in locals():
    ortho_output_dir = output_dir / "orthomosaics"
    ortho_no_gcps_path = ortho_output_dir / "orthomosaic_no_gcps.tif"
    if not ortho_no_gcps_path.exists():
        raise FileNotFoundError(f"Orthomosaic (no GCPs) not found at: {ortho_no_gcps_path.absolute()}")
if 'basemap_esri_path' not in locals():
    basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri.tif")

# Generate visualizations for ESRI comparison
print("Generating visualizations...")
vis_dir = output_dir / "visualizations" / "esri"
vis_dir.mkdir(parents=True, exist_ok=True)

# Load orthomosaic and reference for visualization at reduced resolution to save memory
# Use downsampling to reduce memory usage (load every Nth pixel)
print("  Loading orthomosaic and reference basemap (downsampled for memory efficiency)...")
downsample_factor = 4  # Load every 4th pixel (16x less memory)

def load_downsampled(src_path, downsample=downsample_factor):
    """Load image at reduced resolution to save memory."""
    with rasterio.open(src_path) as src:
        # Read at reduced resolution using windowed reading
        height, width = src.height, src.width
        new_height, new_width = height // downsample, width // downsample
        # Use windowed read with decimation
        window = rasterio.windows.Window(0, 0, width, height)
        data = src.read(1, window=window, out_shape=(new_height, new_width), resampling=rasterio.enums.Resampling.bilinear)
        return data

# Load images one at a time and process visualizations to minimize memory usage
ortho_no_gcps = load_downsampled(ortho_no_gcps_path)
print(f"    Loaded ortho_no_gcps: {ortho_no_gcps.shape} (downsampled from full resolution)")

reference_esri = load_downsampled(basemap_esri_path)
print(f"    Loaded reference_esri: {reference_esri.shape} (downsampled from full resolution)")

# Create visualizations one at a time and clear memory after each
print("  Creating seamline visualizations...")
create_seamline_visualization(
    ortho_no_gcps,
    vis_dir / "seamlines_no_gcps.png",
    title="Seamlines - No GCPs"
)
del ortho_no_gcps

# Reload for error visualizations
print("  Creating error visualizations...")
ortho_no_gcps = load_downsampled(ortho_no_gcps_path)
create_error_visualization(
    ortho_no_gcps, reference_esri,
    vis_dir / "error_no_gcps.png",
    title="Error Map - No GCPs"
)
del ortho_no_gcps, reference_esri

print(f"✓ Visualizations saved to: {vis_dir}")


# OpenStreetMap report
# Load metrics from Step 7 if available, otherwise check for saved files
if 'metrics_no_gcps_osm' not in locals():
    # Try to load from saved JSON files
    if 'metrics_dir' not in locals():
        metrics_dir = comparison_dir / "metrics"
    metrics_file_no_gcps = metrics_dir / "metrics_no_gcps_osm.json"
    
    if metrics_file_no_gcps.exists():
        print(f"Loading saved metrics from: {metrics_dir}")
        if 'json' not in locals():
            import json
        with open(metrics_file_no_gcps, 'r') as f:
            metrics_no_gcps_osm = json.load(f)
        print("✓ Loaded saved OpenStreetMap comparison metrics")
    else:
        raise NameError(
            f"metrics_no_gcps_osm must be defined. "
            f"Please run Step 7 first, or ensure saved metrics exist at:
"
            f"  {metrics_file_no_gcps}"
        )

report_json_osm = output_dir / "quality_report_osm.json"
report_md_osm = output_dir / "quality_report_osm.md"

generate_comparison_report(
    metrics_with_gcps=None,  # Not using GCPs
    metrics_without_gcps=metrics_no_gcps_osm,
    output_path=report_json_osm,
    basemap_source="OpenStreetMap"
)

generate_markdown_report(
    json_report_path=report_json_osm,
    output_path=report_md_osm
)

print(f"
✓ OpenStreetMap report saved:")
print(f"  JSON: {report_json_osm}")
print(f"  Markdown: {report_md_osm}")

# Generate visualizations for OpenStreetMap comparison
print("Generating visualizations for OpenStreetMap...")
vis_dir_osm = output_dir / "visualizations" / "osm"
vis_dir_osm.mkdir(parents=True, exist_ok=True)

# Ensure basemap path is defined
if 'basemap_osm_path' not in locals():
    basemap_osm_path = str(output_dir / "qualicum_beach_basemap_osm.tif")

# Load reference basemap for visualization
print("  Loading reference basemap...")
with rasterio.open(basemap_osm_path) as src:
    reference_osm = src.read(1)  # First band

# Load images at reduced resolution for OpenStreetMap visualizations
print("  Loading orthomosaic for OpenStreetMap visualizations (downsampled)...")
# Reuse the load_downsampled function defined above
ortho_no_gcps = load_downsampled(ortho_no_gcps_path)

# Create visualizations one at a time
print("  Creating seamline visualizations...")
create_seamline_visualization(
    ortho_no_gcps,
    vis_dir_osm / "seamlines_no_gcps.png",
    title="Seamlines - No GCPs"
)
del ortho_no_gcps

# Reload for error visualizations
print("  Creating error visualizations...")
ortho_no_gcps = load_downsampled(ortho_no_gcps_path)
create_error_visualization(
    ortho_no_gcps, reference_osm,
    vis_dir_osm / "error_no_gcps.png",
    title="Error Map - No GCPs"
)
del ortho_no_gcps, reference_osm

print(f"✓ Visualizations saved to: {vis_dir_osm}")

# Generate comprehensive LaTeX/PDF report with both ESRI and OSM
# Check for shifted metrics (from Step 7.5) and GCP-aligned metrics (from Step 7.6)
metrics_dir = comparison_dir / "metrics"
shifted_metrics_no_gcps_path = metrics_dir / "metrics_shifted_no_gcps_esri.json"
gcp_aligned_metrics_no_gcps_path = metrics_dir / "metrics_gcp_aligned_no_gcps_esri.json"

print("
" + "=" * 60)
print("Generating comprehensive LaTeX/PDF report (ESRI + OSM, NO GCPs only)...")
print("=" * 60)

report_latex_final = output_dir / "quality_report_final"
latex_result_final = generate_latex_report(
    json_report_path=report_json_esri,
    output_path=report_latex_final,
    visualization_dir=vis_dir,
    json_report_path_osm=report_json_osm,
    visualization_dir_osm=vis_dir_osm,
    shifted_metrics_no_gcps_path=shifted_metrics_no_gcps_path if shifted_metrics_no_gcps_path.exists() else None,
    shifted_metrics_with_gcps_path=None,  # Not using GCPs
    gcp_aligned_metrics_no_gcps_path=gcp_aligned_metrics_no_gcps_path if gcp_aligned_metrics_no_gcps_path.exists() else None,
    gcp_aligned_metrics_with_gcps_path=None  # Not using GCPs
)

if latex_result_final.suffix == '.pdf':
    print(f"
✓ PDF report generated: {latex_result_final}")
else:
    print(f"
✓ LaTeX file generated: {latex_result_final}")
    print("  (Install LaTeX and run: pdflatex quality_report_final.tex to generate PDF)")


## Step 9: Display Report Summary


In [ ]:
# Display summary from ESRI report
# Ensure imports and variables are defined
if 'Path' not in locals():
    from pathlib import Path
if 'json' not in locals():
    import json

if 'output_dir' not in locals():
    output_dir = Path("outputs")

# Define report paths
report_json_esri = output_dir / "quality_report_esri.json"
report_json_osm = output_dir / "quality_report_osm.json"
report_md_esri = output_dir / "quality_report_esri.md"
report_md_osm = output_dir / "quality_report_osm.md"

print("=" * 60)
print("QUALITY COMPARISON SUMMARY (ESRI World Imagery)")
print("=" * 60)

with open(report_json_esri, 'r') as f:
    report_esri = json.load(f)

comparison = report_esri.get('comparison', {})

if comparison.get('rmse_improvement'):
    rmse = comparison['rmse_improvement']
    print(f"\nRMSE Improvement: {rmse['percentage']:+.2f}%")
    print(f"  Without GCPs: {rmse['without_gcps']:.4f}")
    print(f"  With GCPs:    {rmse['with_gcps']:.4f}")

if comparison.get('mae_improvement'):
    mae = comparison['mae_improvement']
    print(f"\nMAE Improvement: {mae['percentage']:+.2f}%")
    print(f"  Without GCPs: {mae['without_gcps']:.4f}")
    print(f"  With GCPs:    {mae['with_gcps']:.4f}")

if comparison.get('similarity_improvement'):
    sim = comparison['similarity_improvement']
    print(f"\nSimilarity Improvement: {sim['percentage']:+.2f}%")
    print(f"  Without GCPs: {sim['without_gcps']:.4f}")
    print(f"  With GCPs:    {sim['with_gcps']:.4f}")

if comparison.get('seamline_reduction'):
    seam = comparison['seamline_reduction']
    print(f"\nSeamline Reduction: {seam['percentage']:+.2f}%")
    print(f"  Without GCPs: {seam['without_gcps']:.2f}%")
    print(f"  With GCPs:    {seam['with_gcps']:.2f}%")

print("\n" + "=" * 60)
print(f"Full reports available at:")
print(f"  {report_md_esri}")
print(f"  {report_md_osm}")
print("=" * 60)
